# Changing Functions of the Taxes and Transfers System

This tutorial focuses on the policy functions of GETTSIM. Alongside policy parameters, these functions help GETTSIM define a date-specific policy environment based on which it can compute taxes and transfers for individual and household data. 

Just like [parameters](parameters.ipynb), policy functions can be replaced, added or removed to make changes to the existing policy environment. This way, you can design a new tax or transfer for any specific group of people, e.g. invent a new tax for people that have income from renting an apartment, or change the conditions for receiving already existing transfers.

This tutorial showcases the policy functions using a concrete example. For a more comprehensive and abstract discussion of the feature, check out the [how-to guide on Different Ways to Load Policy Functions](../how_to_guides/different_ways_to_load_policy_functions.ipynb).


In [ ]:
import warnings

import numpy as np
import plotly.express as px

from gettsim import (
    FunctionsAndColumnsOverlapWarning,
    compute_taxes_and_transfers,
    create_synthetic_data,
    set_up_policy_environment,
)

warnings.filterwarnings("ignore", category=FunctionsAndColumnsOverlapWarning)

## Changing  and Replacing Existing Function(s)

### Example: Receiving Multiple Transfers

In the German system, there are some transfers for low-income families that cannot be received in combination. Per default, GETTSIM will always choose the most favorable transfers and set other transfers to zero. This assumption could model the behavior of households/families in a wrong way, if they do not always choose the optimal transfers (from a monetary perspective). For example, there could be a social stigma connected to certain transfers or some people simply do not know about some of the available transfers.

To account for these frictions, we can turn off this aspect of GETTSIM so that we see all the transfers a family/household is entitled to, even if the transfers cannot be received in combination. This can be useful for further analysis. For example you could speculate which transfers Germans receive in reality and implement this in GETTSIM.

### Find the Function

[Here](../gettsim_objects/functions.rst) we can look for the function that implements the aspect we want to change.

In [ ]:
environment = set_up_policy_environment(2020)

### Define Changes to the Function

After you found the function that you want to change, copy the source code from the website to your notebook and change it just as you like:

In [ ]:
def arbeitslosengeld_2__betrag_m_bg(
    arbeitslosengeld_2__anspruchshöhe_m_bg,
    # vorrangpruefungen__wohngeld_vorrang_bg,
    # vorrangpruefungen__kinderzuschlag_vorrang_bg,
    # vorrangpruefungen__wohngeld_kinderzuschlag_vorrang_bg,
    erwachsene_alle_rentner_hh,
):
    if (
        # vorrangpruefungen__wohngeld_vorrang_bg
        # | vorrangpruefungen__kinderzuschlag_vorrang_bg
        # | vorrangpruefungen__wohngeld_kinderzuschlag_vorrang_bg
        erwachsene_alle_rentner_hh
    ):
        out = 0.0
    else:
        out = arbeitslosengeld_2__anspruchshöhe_m_bg

    return out

The lines of the cell above that start with "#" usually do the priority check as described above. With the hash, the lines become a comment and do not influence the code anymore.

### Make GETTSIM Incorporate your Changes 

The `PolicyEnvironment` offers dedicated methods to change policy functions. In this case, we can use the function `upsert_functions` (update/insert) to alter the existing policy environment. This function returns a new `PolicyEnvironment` object with the updated functions. Note that the original `PolicyEnvironment` object remains unchanged, so we need to assign the new object to a variable (below this is `environment_no_check`).

In [ ]:
environment_no_check = environment.upsert_functions(arbeitslosengeld_2__betrag_m_bg)

Computations with the new `environment_no_check` will now have the characteristic of showing the value of all available transfers without checking which ones cannot be received in combination and without choosing the most profitable combination. 

Let´s test if this works! 

We import simulated data for households with two parents and two children. These households only vary in their income:

In [ ]:
## idea for use of synthetical data
data = create_synthetic_data(
    n_adults=2,
    n_children=2,
    specs_heterogeneous={
        "bruttolohn_m": [[i, 0, 0, 0] for i in np.linspace(500, 5000, 250)]
    },
)

# Compute sum of pension contributions in household and add it to data.
rente__altersrente__sum_private_gesetzl_rente_m = compute_taxes_and_transfers(
    data=data,
    environment=environment,
    targets="rente__altersrente__sum_private_gesetzl_rente_m",
)

data["rente__altersrente__sum_private_gesetzl_rente_m"] = (
    rente__altersrente__sum_private_gesetzl_rente_m[
        "rente__altersrente__sum_private_gesetzl_rente_m"
    ]
)
data.head(5)

For this data we can now compare the results of using GETTSIM with the `environment_no_check` and the usual `environment`. 

We should expect to see positive values for `wohngeld__betrag_m_wthh`, `kinderzuschlag__betrag_m_bg` and
`arbeitslosengeld_2__betrag_m_bg` at the same time if we do not check which combination of
transfers is optimal (`environment_no_check`).

On the other hand, if we use the default version of the `environment`,
`wohngeld__betrag_m_wthh` and `kinderzuschlag__betrag_m_bg` should be zero as long as
`arbeitslosengeld_2__betrag_m_bg` is positive (and the other way around), as it is a characteristic
of the German taxes and transfers system that *Wohngeld* and *Kinderzuschlag* cannot be
received in combination with *Arbeitslosengeld 2*.

In [ ]:
targets = [
    "wohngeld__betrag_m_wthh",
    "kinderzuschlag__betrag_m_bg",
    "arbeitslosengeld_2__betrag_m_bg",
]

In [ ]:
environments = {
    "Checked Favorability": environment,
    "No Check of Favorabilty": environment_no_check,
}

In [ ]:
# Loop through keys to plot both scenarios.
for k in environments:
    # Compute taxes and transfers.
    result = compute_taxes_and_transfers(
        data=data,
        environment=environments[k],
        targets=targets,
    )
    # Add earnings and index to result DataFrame.
    result["bruttolohn_m"] = data["bruttolohn_m"]
    result.index = data["hh_id"]
    # Create DataFrame that contains the maximum value of the target variables
    # in the household and the household gross income.
    result = (
        result.groupby("hh_id")[targets]
        .max()
        .join(result.groupby("hh_id")["bruttolohn_m"].sum())
    )
    # Plot the results.
    fig = px.line(
        data_frame=result,
        x="bruttolohn_m",
        y=targets,
        title=k,
    )
    fig.update_layout(
        xaxis_title="Monthly gross income in € (per household)",
        yaxis_title="€ per month",
    )
    fig.show()

On first glance, both figures look quite confusing because of the complexity of the German taxes and transfers system. But if we take a closer look, the figures confirm our expectations. If we let GETTSIM check for the most favorable combination of transfers, ```wohngeld__betrag_m_wthh``` and ```kinderzuschlag__betrag_m_bg``` are zero as long as `arbeitslosengeld_2__betrag_m_bg` is positive (i.e. the best option for the household) and the other way around. 

If we do not let GETTSIM do this check, this does not hold any longer and all transfers can be positive at the same time (which is what we were trying to achieve). 


### Multiple Functions

You can use exactly the same approach if you want to change more than one function of GETTSIM. But first, for our example we need to invent some changes to another function of GETTSIM. Imagine, we want to double the amount of *Kindergeld* every household receives in addition to the previously implemented function change.

In [ ]:
def kindergeld__betrag_m(
    kindergeld__anzahl_ansprüche: bool,
    kindergeld_params: dict,
) -> float:
    """Sum of Kindergeld for eligible children.

    Kindergeld claim for each child depends on the number of children Kindergeld is
    being claimed for.

    Parameters
    ----------
    kindergeld__anzahl_ansprüche
        See :func:`kindergeld__anzahl_ansprüche`.
    kindergeld_params
        See params documentation :ref:`kindergeld_params <kindergeld_params>`.

    Returns
    -------

    """

    if kindergeld__anzahl_ansprüche == 0:
        sum_kindergeld = 0.0
    else:
        sum_kindergeld = sum(
            kindergeld_params["kindergeld"][
                (min(i, max(kindergeld_params["kindergeld"])))
            ]
            for i in range(1, kindergeld__anzahl_ansprüche + 1)
        )

    return sum_kindergeld * 2

If you edit ```arbeitslosengeld_2__betrag_m_bg``` and ```kindergeld__betrag_m```, your way to make GETTSIM incorporate your changes would be:

In [ ]:
df = compute_taxes_and_transfers(
    data=data,
    environment=environment.upsert_functions(
        arbeitslosengeld_2__betrag_m_bg, kindergeld__betrag_m
    ),
    targets=[
        "wohngeld__betrag_m_wthh",
        "kinderzuschlag__betrag_m_bg",
        "arbeitslosengeld_2__betrag_m_bg",
        "kindergeld__betrag_m",
    ],
)

## Adding a New Function

Instead of replacing existing functions, we can similarly define completely new functions and add them to the policy environment.

## Aggregation Functions

Functions that aggregate columns are treated differently in GETTSIM. Column can be
aggregated based on a group ID (e.g. on the household level) or based on the individual
identifier `p_id`.

### Aggregation Based on Group ID


If we would like to add (or replace) such functions, we need to specify them in a
dictionary which we provide to `compute_taxes_and_transfers` via the
`aggregate_by_group_specs_tree` argument. An example dictionary is as follows:


```python
aggregate_by_group_specs_tree = {
    "anz_erwachsene_sn": {"source_col": "erwachsen", "aggr": "sum"},
    "anz_personen_hh": {"aggr": "count"},
}
```

### Aggregation Based on `p_id`

Similarly to above, aggregation functions based on `p_id` are specified in a dictionary
which can be provided to `compute_taxes_and_transfers` via the `aggregate_by_p_id_specs_tree`
argument. An example dictionary is as follows:

```python
aggregate_by_p_id_kindergeld = {
    "kindergeld__anzahl_ansprüche": {
        "p_id_to_aggregate_by": "p_id_kindergeld_empf",
        "source_col": "kindergeld__anspruchsberechtigt",
        "aggr": "sum",
    },
}
```

See [GEP 4](https://gettsim.readthedocs.io/en/stable/geps/gep-04.html) for more
information on aggregation functions.